In [112]:
import json
import pandas
import time
import os
import recordlinkage
import jellyfish as jf
import csv

In [113]:
outputDir = 'scraped_json//'
field_agg = "_"

## Load the source file dict
sourceFiles = json.loads(open('sourceFiles.json', "r").read())

## Load the file schema
sourceSchema = json.loads(open('sourceSchema.json', "r").read())

## Load the id config
idConfig = json.loads(open('idConfig.json', "r").read())

In [114]:
def mergeSourceFiles (source, outputDir, sourceFiles):
    recordList = []
    for file in sourceFiles[source]:  
        file = json.loads(open(outputDir + file, "r", encoding="utf-8").read())
        for record in file:
            recordList.append(record)
    return recordList

In [115]:
## Cycle through all of files per source and drop into a list of dicts
## Printing list names for reference sake
for key in sourceFiles.keys():
    print(key)
    vars()[key] = mergeSourceFiles(key, outputDir, sourceFiles)
    

sports247
rivals
cfbRef
allConf
allAmerican
nflData


In [116]:
## Clean up dirty names
def mungeID(playerString):
    return ''.join(e for e in playerString if e.isalnum()).lower().replace("jr.", "").replace("st.", "state")  

In [117]:
#create a universal identifier
#CHANGE - this is not going to be a list
#CFB-Ref reports it in chronological order so we will join based on position 1

def createNewID (fieldList, thisDict, fieldAgg):
    finalID= ''
    for i in thisDict:
        for idx, val in enumerate(fieldList):
            if (type(i[val]) is list):
                i[val]= mungeID(i[val][0])
                if (len(fieldList) -1 == idx):
                    finalID += str(i[val]).strip('[]').strip("''")
            elif (type(val) is not list):
                i[val] = mungeID(i[val])
                if (len(fieldList) - 1 == idx):
                    finalID += i[val]
                else:
                    finalID = i[val] + fieldAgg
        i['ID'] = finalID
        finalID=''

In [118]:
createNewID(idConfig['sports247'], sports247, field_agg)
createNewID(idConfig['rivals'], rivals, field_agg)
createNewID(idConfig['cfbRef'], cfbRef, field_agg)
createNewID(idConfig['allConf'], allConf, field_agg)
createNewID(idConfig['allAmerican'], allAmerican, field_agg)
createNewID(idConfig['nflData'], nflData, field_agg)

#createNewID(idConfig['rivals'], rivals, field_agg)

In [119]:
df_sports247 = pandas.DataFrame(sports247)
df_rivals = pandas.DataFrame(rivals)
df_cfbRef = pandas.DataFrame(cfbRef)
df_allConf = pandas.DataFrame(allConf)
df_allAmerican = pandas.DataFrame(allAmerican)
df_nflData = pandas.DataFrame(nflData)

#df_sports247.dtypes
df_rivals.dtypes

school           object
year             object
playerName       object
city             object
state            object
highSchool       object
position         object
height          float64
weight          float64
stars             int64
nationalRank    float64
positionRank    float64
stateRank       float64
ID               object
dtype: object

In [108]:
print(rivals)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [120]:
indexer = recordlinkage.Index()
indexer.block('school')
indexer.block('year')
candidate_links = indexer.index(df_sports247, df_rivals)

In [122]:
df = pandas.DataFrame(candidate_links)
df

,0
0,"(0, 19)"
1,"(0, 34)"
2,"(0, 43)"
3,"(0, 51)"
4,"(0, 55)"
...,...
22388443,"(17063, 16853)"
22388444,"(17063, 16855)"
22388445,"(17063, 16856)"
22388446,"(17063, 16861)"


In [124]:
c = recordlinkage.Compare()

c.string('playerName', 'playerName', method='levenshtein', threshold=3)

# The comparison vectors
feature_vectors = c.compute(candidate_links, df_sports247, df_rivals)

pandas.DataFrame(feature_vectors)

0
0     19     0.0
      34     0.0
      43     0.0
      51     0.0
      55     0.0
...          ...
17063 16853  0.0
      16855  0.0
      16856  0.0
      16861  0.0
      16877  0.0

[22388448 rows x 1 columns]

In [125]:
ecm = recordlinkage.ECMClassifier()
ecm.fit_predict(feature_vectors)

MultiIndex([], )

In [126]:
print(ecm)

In [127]:
ecm[0]

TypeError: 'ECMClassifier' object is not subscriptable